#**Llama 2+ Pinecone + LangChain**

##**Step 1: Install All the Required Libraries**

In [ ]:
!pip install langchain
!pip install pypdf
!pip install unstructured
!pip install sentence_transformers
!pip install pinecone-client
!pip install llama-cpp-python
!pip install huggingface_hub

#**Step 2: Import All the Required Libraries**

In [2]:
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer
from langchain.chains.question_answering import load_qa_chain
import pinecone
import os

#**Step 3: Load the Data- Download the Pdf you want to work with and place it in working directory**

In [3]:
loader = PyPDFLoader("sample_election.pdf")

In [4]:
data = loader.load()

In [7]:
data[1]

Document(page_content='4.17 \uf0d8                  Delimitation of constituencies  \n4.18 \uf0d8                  Defections  \n4.19 \uf0d8                  Oversized Council of Ministers  \n4.20 \uf0d8                  Problems of Instability  \n4.21 \uf0d8                  The eligibility of non -Indian born citizens to hold high offices  \n4.22 \uf0d8                  Appointment of the Election Commissioners and State Election \nCommissioners  \n4.23 \uf0d8                  Candidates owing Government dues  \n4.24 \uf0d8                  Office of profit  \n  \n4.25 B.  Political Parties  \n\uf0d8                  Background and  objectives  \n4.26 \uf0d8                  Constitution and Law in relation to Political Parties  \n4.27 \uf0d8                  The present scenario  \n4.28 \uf0d8                  Studies and reports  \n4.29 \uf0d8                  Identifying the problem areas  \n4.30 \uf0d8                  Law for political parties  \n4.31 \uf0d8                  Rec

#**Step 4: Split the Text into Chunks**

In [15]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [16]:
docs=text_splitter.split_documents(data)

In [17]:
len(docs)

85

In [18]:
docs[10]

Document(page_content='the process of political mobilization of group identities on non -ideological lines.  \n\uf09b   An ineffective and slow judicial process of dealing with election petitions, \nrendering the whole process quite often mea ningless.  \n\uf09b   Fake and non -serious candidates who create major practical difficulties and are \nalso used indirectly to subvert the electoral process.  \n\uf09b   Incongruities in delimitation of constituencies resulting in poor representation.  \n\uf09b   Problems of insta bility, hung legislative houses and their relation to the electoral \nlaws and processes.  \n\uf09b   Last but not the least, loss of systemic legitimacy due to decay in the standards \nof political morality and decline in the spirit of service and sacrifice in public  life. \n  \nAttempts at Reforms', metadata={'source': 'sample_election.pdf', 'page': 5})

#**Step 5: Setup the Environment**

In [32]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = #put your hugging face api token here
os.environ["PINECONE_API_KEY"]= #replace this with your pinecone api key, find it in your pinecone account
os.environ["PINECONE_API_ENV"]= #you will find the pinecone api env in your pinecone account details

In [ ]:
PINECONE_API_KEY = ''#replace this with your pinecone api key, find it in your pinecone account
PINECONE_API_ENV = ''#you will find the pinecone api env in your pinecone account details

#**Step 6: Downlaod the Embeddings**

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

#**Step 7: Initializing the Pinecone**

In [36]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = 'llama2-practice' # put in the name of your pinecone index here

In [43]:
docs[84]

Document(page_content='aggregative mechanism in major parties and curb the tendency of regionalisation and \nfragmentation.  If the national convention is found too expensive, a series of staggered \nState conventions may be held by major parties wi th their respective slates of \ncandidates (common for all the States in one party).  This will go a long way in making \nparties aggregative and thus more functional in a parliamentary federal system of \ngovernance.', metadata={'source': 'sample_election.pdf', 'page': 31})

#**Step 8: Create Embeddings for Each of the Text Chunk**

In [47]:
docsearch=Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)

# If you already have an index, you can load it like this


In [ ]:
#docsearch = Pinecone.from_existing_index(index_name, embeddings)

#**Step 9: Similarity Search**

In [48]:
query="Statistically, what is the number of voters in India?"

In [49]:
docs=docsearch.similarity_search(query)

In [51]:
docs[0]

Document(page_content='of elections in an independent Election Commission.  Under articles 243K and 243ZA \nelections to local bodies – Panchayats and Municipalities – are the responsibility of \nState Electio n Commissions.  \n  \nMagnitude of the Task  \n  \n4.3 A general election to Lok Sabha  is a gigantic exercise.  It has been said that \nholding general elections in India is equal to holding them for Europe, United States, \nCanada and Australia all put together.  Statistically, the number of voters in India is in \nexcess of 600 million (60 cror es).  The number of polling booths all over the country \nadds up to about 900,000 (9 lakhs).  To manage these polling booths about five million', metadata={})

In [62]:
query="what is booth capturing and rigging?"

In [63]:
docs=docsearch.similarity_search(query)
docs

[Document(page_content='complaints of large scale rigging.  This also makes counting easy and non -contestable \nand theoretically the results could be available within a very short time if the system \nwanted to make prompt announcement of the winning candidates.   \n  \nThe Commission recommends the introduction of EVMs in all constituencies all \nover the country for all elections as rapidly as possible.  \n  \nBooth Capturing and Rigging  \n  \n4.10 On the question of booth capturing, rigging and intimidation of voters, the \nproposals of the Goswami Committee to the effect that EC should be empowered to \ntake more stringent action should be accepted and implemented in full.', metadata={}),
 Document(page_content='first-past-the-post system and the multiplicity of parties and large number of \nindependents.  \n\uf09b   The question of  defections and the Tenth Schedule.  \n\uf09b   Inaccurate and flawed electoral rolls and voter identity leading to rigging and \ndenial of voting r

#**Step 9: Query the Docs to get the Answer Back (Llama 2 Model)**

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose

#Import All the Required Libraries

In [64]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download
from langchain.chains.question_answering import load_qa_chain

In [65]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

#  Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGLM library.

We can see the different variations that Llama-2-13B-GGML has [here](https://huggingface.co/models?search=llama%202%20ggml).



In this case, we will use the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).

 Quantization reduces precision to optimize resource usage.

Quantization is a technique to reduce the computational and memory costs of running inference by representing the weights and activations with low-precision data types like 8-bit integer ( int8 ) instead of the usual 32-bit floating point ( float32 ).

In [66]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [67]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [109]:
model_path

'/Users/admin/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/47d28ef5de4f3de523c421f325a2e4e039035bab/llama-2-13b-chat.ggmlv3.q5_1.bin'

In [101]:
n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 256  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Loading model,
llm = LlamaCpp(
    model_path=model_path,
    max_tokens=256,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    n_ctx=1024,
    verbose=False,
)

llama.cpp: loading model from /Users/admin/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/47d28ef5de4f3de523c421f325a2e4e039035bab/llama-2-13b-chat.ggmlv3.q5_1.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 1024
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 9 (mostly Q5_1)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_int

In [102]:
chain=load_qa_chain(llm, chain_type="stuff")

In [70]:
query="what are electoral rolls?"
docs=docsearch.similarity_search(query)

In [71]:
docs

[Document(page_content='subordinate legislation and executive action.  The Commission is happy to note that \nsome of the suggestions mentioned in the consultation papers released by it are already \nunder implementation.  \n  \nElectoral Rolls and Voter ID  \n  \n4.8.1  The electoral process beg ins with the preparation of electoral rolls. If the rolls are \nincomplete or defective, the whole electoral process is vitiated.  Any serious attempt at \nelectoral reform, therefore, must first tackle the question of faulty electoral rolls. At \npresent, the E lection Commission (EC) is responsible for preparing the electoral rolls for \nassembly and parliamentary constituencies.  The State Election Commissions are \nresponsible for electoral rolls for local body elections.   In some States, the EC and \nState Electio n Commissions (SECs) have agreed to coordinate the preparation of \nelectoral rolls.  \n  \n4.8.2  Given the technology today, an automated and well designed  online system', m

In [72]:
chain.run(input_documents=docs, question=query)

 Electoral Rolls are lists of eligible voters that are prepared by the Election Commission (EC) for assembly and parliamentary constituencies, with the help of State Election Commissions (SECs) in some states. The rolls should be updated constantly and periodically posted

' Electoral Rolls are lists of eligible voters that are prepared by the Election Commission (EC) for assembly and parliamentary constituencies, with the help of State Election Commissions (SECs) in some states. The rolls should be updated constantly and periodically posted'

In [73]:
query="Summarize the problems faced by the commission in 5 points."
docs=docsearch.similarity_search(query)

In [74]:
docs

[Document(page_content='poll. There is no revision of electoral rolls, no fresh nominations, no fresh campaigning \nor the like. The Commission is of the view that there  are substantial advantages of \nfollowing the  policy of 50% plus one vote. On the one hand, it resolves the problem of \ninadequate representation.  On the other, it also makes it in the self -interest of various \npolitical parties themselves to widen their app eal to a wider electorate.  It can help push \npolitical rhetoric in a direction of mobilizing language might take on comparative \n„universal‟ tone as opposed to „sectoral‟ tones of the present day. With the need to be \nmore broad -based in their appeal, issues  that have to do with good governance rather \nthan with cleavages and narrow identities might start to surface in the country.  \n  \n4.16.5 Despite the suggested merits of this system, the Commission refrains from \nmaking a positive recommendation for its accep tance straightaway, as the Commissio

In [75]:
chain.run(input_documents=docs, question=query)

 The problems are as follows:

Increasing cost of elections leading to unethical and illegal funding.
Competing role expectations and conflict of perceptions between directly elected representatives from different levels.
Representative character of representatives becoming doubtful due to the first-past-the-post system.
Lack of involvement of electorate in the selection of candidates.
Need for reforms in the political party system to make electoral reforms effective.

' The problems are as follows:\n\nIncreasing cost of elections leading to unethical and illegal funding.\nCompeting role expectations and conflict of perceptions between directly elected representatives from different levels.\nRepresentative character of representatives becoming doubtful due to the first-past-the-post system.\nLack of involvement of electorate in the selection of candidates.\nNeed for reforms in the political party system to make electoral reforms effective.'

<font size="6">Query</font>

In [103]:
query="What is winning by minority vote? explain in detail by examples"
docs=docsearch.similarity_search(query)

<font size="6">Matching Document</font>

In [106]:
docs[0]

Document(page_content='Question of Representational Legitimacy  \n  \n4.16.1   Winning by minority vote : The multiplicity of political parties, combined with our \nWestminster based first -past-the-post system results in a majority of legislators and \nparliamentarians getting elected on a minority vote.  In other words, they usually win by \nobtaining less than 5 0% of the votes cast, i.e. with more votes cast against them than in \ntheir favour. There are States where 85% to 90% of the legislators have won on a \nminority vote.  At the national level, the proportion of MPs who have won on a minority \nvote is over 67% a t an average for the last three Lok Sabha elections. In extreme \ncases, some candidates have won even on the basis of 13% of the votes polled. (For \nrelevant dat a, see Tables in Annexures to the Consultation Paper on the subject).  \n  \n4.16.2 In a pluralistic s ociety such as ours, some political parties have found it', metadata={})

<font size="6">Chat output</font>

In [105]:
chain.run(input_documents=docs, question=query)


Winning by a minority vote means receiving fewer than half of the total votes cast and yet being declared the winner. In other words, if more people voted against the winner than for them, they still win because of how the electoral system is set up. This is what occurs frequently in India's first-past-the-post system, where candidates need not receive a majority (more than 50%) of the votes to be elected. In extreme cases, some winners have received as little as 13% of the vote. This situation has serious implications for democratic representation because it indicates that many citizens do not support their representatives, which can lead to an unequal and unrepresentative political system.

"\nWinning by a minority vote means receiving fewer than half of the total votes cast and yet being declared the winner. In other words, if more people voted against the winner than for them, they still win because of how the electoral system is set up. This is what occurs frequently in India's first-past-the-post system, where candidates need not receive a majority (more than 50%) of the votes to be elected. In extreme cases, some winners have received as little as 13% of the vote. This situation has serious implications for democratic representation because it indicates that many citizens do not support their representatives, which can lead to an unequal and unrepresentative political system."

In [107]:
query="Tell me about EVMs."
docs=docsearch.similarity_search(query)
docs

[Document(page_content='should be allowed to be inspected on payment of a nominal fee by anyone.  \nFacilities should a lso be provided to the members of the public at the post offices \nfor submitting their applications for modification of the electoral rolls.  \n  \n  \n  \n  \n  \nElectronic Voting Machines (EVMs)  \n  \n4.9 EVMs have been used successfully in limited areas in sensitive co nstituencies \nand in bye -elections.  Any doubts about this technology were amply removed during the \ndeliberations of the Goswami Committee.  Not only were the EVMs satisfactorily \ndemonstrated to all politician members, several electronic experts of the Gove rnment of \nIndia also testified that the machines could be used at all our elections without any \nmisgivings.  The advantages of EVMs in preventing large -scale rigging are quite \napparent as the machine locks up and will permit only one hit every so many seco nds.  \nWherever these EVMs have been used in urban and in rural areas, ther

In [108]:
chain.run(input_documents=docs, question=query)

 EVMs are electronic voting machines that have been used successfully in limited areas in sensitive constituencies and bye-elections. The advantages of EVMs in preventing large-scale rigging are quite apparent, as the machine locks up and will permit only one hit every so many seconds. There have been no complaints of large-scale rigging wherever these EVMs have been used in urban and rural areas. The Commission recommends the introduction of EVMs in all constituencies all over the country for all elections as rapidly as possible.

' EVMs are electronic voting machines that have been used successfully in limited areas in sensitive constituencies and bye-elections. The advantages of EVMs in preventing large-scale rigging are quite apparent, as the machine locks up and will permit only one hit every so many seconds. There have been no complaints of large-scale rigging wherever these EVMs have been used in urban and rural areas. The Commission recommends the introduction of EVMs in all constituencies all over the country for all elections as rapidly as possible.'